In [6]:
import sqlite3

print(sqlite3.sqlite_version)

3.39.3


In [7]:
conn = sqlite3.connect('pythonjsondemo.db') # permanent database

In [8]:
cursor = conn.cursor()

### Using requests to get JSON data from a web site.
https://www.w3schools.com/python/module_requests.asp

If you don't have the requests module installed just use pip as shown below

***pip install requests***

In [9]:
import requests

countries_api_res = requests.get('http://api.worldbank.org/countries?format=json&per_page=100')
countries = countries_api_res.json()[1]

In [10]:
import pprint

pprint.pprint(countries[0])

{'adminregion': {'id': '', 'iso2code': '', 'value': ''},
 'capitalCity': 'Oranjestad',
 'id': 'ABW',
 'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
 'iso2Code': 'AW',
 'latitude': '12.5167',
 'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
 'longitude': '-70.0167',
 'name': 'Aruba',
 'region': {'id': 'LCN',
            'iso2code': 'ZJ',
            'value': 'Latin America & Caribbean '}}


In [11]:
cursor.executescript('''

DROP TABLE IF EXISTS countries;

CREATE TABLE countries (id varchar(3), data json)
''');

## Inserting the JSON into a SQLite table

Note:  json is included in the standard Python distibution

### json.dumps() 

Serialize obj to a JSON formatted str using this conversion table. The arguments have the same meaning as in dump().

#### Documentation
https://docs.python.org/3/library/json.html

In [12]:
import json

for country in countries:
    cursor.execute("insert into countries values (?, ?)",
                    [country['id'], json.dumps(country)])
    conn.commit()

## Query the data using select

In [15]:
cursor.execute('''select json_extract(data, '$.name', '$.iso2Code', '$.id') from countries;''').fetchmany(8)

[('["Aruba","AW","ABW"]',),
 ('["Africa Eastern and Southern","ZH","AFE"]',),
 ('["Afghanistan","AF","AFG"]',),
 ('["Africa","A9","AFR"]',),
 ('["Africa Western and Central","ZI","AFW"]',),
 ('["Angola","AO","AGO"]',),
 ('["Albania","AL","ALB"]',),
 ('["Andorra","AD","AND"]',)]

In [16]:
x = cursor.execute('''select json_extract(data, '$.name', '$.iso2Code', '$.id') from countries;''').fetchmany(8)
type(x)

list

## Query the JSON data using pandas

In [17]:
import sqlite3
import pandas as pd

pd.read_sql_query('''
select json_extract(data, '$.name', '$.iso2Code', '$.id') as countrydata
from countries
limit 3;''', conn)

,countrydata
0,"[""Aruba"",""AW"",""ABW""]"
1,"[""Africa Eastern and Southern"",""ZH"",""AFE""]"
2,"[""Afghanistan"",""AF"",""AFG""]"


## Query each JSON attribute as a separate column

In [18]:
import sqlite3
import pandas as pd

pd.read_sql_query('''
select json_extract(data, '$.name') as name,
json_extract(data, '$.id') as id,
json_extract(data, '$.iso2Code') as iso2Code 
from countries
order by iso2code
limit 5;''', conn)

,name,id,iso2Code
0,Arab World,ARB,1A
1,East Asia & Pacific (excluding high income),EAP,4E
2,Europe & Central Asia (excluding high income),ECA,7E
3,Africa,AFR,A9
4,Andorra,AND,AD


## Exploring the JSON data

In [14]:
cursor.execute('''SELECT countries.rowid, fullkey, value
  FROM countries, json_tree(countries.data) 
''' ).fetchmany(10)

[(1,
  '$',
  '{"id":"ABW","iso2Code":"AW","name":"Aruba","region":{"id":"LCN","iso2code":"ZJ","value":"Latin America & Caribbean "},"adminregion":{"id":"","iso2code":"","value":""},"incomeLevel":{"id":"HIC","iso2code":"XD","value":"High income"},"lendingType":{"id":"LNX","iso2code":"XX","value":"Not classified"},"capitalCity":"Oranjestad","longitude":"-70.0167","latitude":"12.5167"}'),
 (1, '$.id', 'ABW'),
 (1, '$.iso2Code', 'AW'),
 (1, '$.name', 'Aruba'),
 (1,
  '$.region',
  '{"id":"LCN","iso2code":"ZJ","value":"Latin America & Caribbean "}'),
 (1, '$.region.id', 'LCN'),
 (1, '$.region.iso2code', 'ZJ'),
 (1, '$.region.value', 'Latin America & Caribbean '),
 (1, '$.adminregion', '{"id":"","iso2code":"","value":""}'),
 (1, '$.adminregion.id', '')]

## Close the database connection and cursor

In [15]:
cursor.close()
conn.close()